In [8]:
import numpy as np
from steps import *
from tqdm import tqdm
from optimal import *
import matplotlib.pyplot as plt
import optuna
import pickle
import warnings
warnings.filterwarnings("error")


In [9]:
def plot_res(x_axis, results, x_label, y_label, label, title, path):
    plt.grid()
    plt.plot(x_axis, results.mean(axis=1), label=label)
    plt.fill_between(x_axis, results.mean(axis=1)+results.std(axis=1), results.mean(axis=1)-results.std(axis=1),
                     color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig(path+".png")
    plt.clf()

    plt.grid()
    plt.boxplot(results.T, positions=x_axis)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig("BP-"+path+".png")
    plt.clf()

In [10]:
d = 20
opt_r = 3
m = 20
T_r = 100
times = 10
opt_sigma = 5
sigma_type = matrix_type.DIAG
s_type = matrix_type.IDENTITY

name = f"optuna_optimization_d={d}_r={opt_r}_sigma={sigma_type}_s={s_type}_m={m}_Tr={T_r}_var={opt_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)

[I 2023-05-13 23:43:52,770] Using an existing study with name 'optuna_optimization_d=20_r=3_sigma=matrix_type.DIAG_s=matrix_type.IDENTITY_m=20_Tr=100_var=5' instead of creating a new one.


In [11]:
# Sigma
sigma_max = 10.0
sigma_min = 0.5
spaces = 0.5
load = False

suffix = f'{sigma_min}-{sigma_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    d_sigmas = np.linspace(sigma_min, sigma_max, int((sigma_max-sigma_min)/spaces)+1)
    results = np.zeros([len(d_sigmas), times])
    for idx, d_sigma in enumerate(d_sigmas):
        print(f" ***** Sigma={d_sigma} *****")
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_sigma0_{suffix}.p', 'wb'))
    pickle.dump(d_sigmas, open(f'sigma0-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_sigma0_{suffix}.p', 'rb'))
    d_sigmas = pickle.load(open(f'sigma0-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_sigmas, results=results,
         x_label=r'$\sigma_0$', y_label='Accuracy Ratio', label=f'd={d}, r={opt_r}',
         title=r'results accuracy ratio vs $\sigma_0$',
         path=f"results_accuracy_ratio_sigma0_{suffix}")

 ***** Sigma=0.5 *****
l_star=9 ; regret_mix=0.8732707472125424 ; last_regret=1.125902920729875
l_star=12 ; regret_mix=0.8857670755587402 ; last_regret=1.0408081123893682
l_star=10 ; regret_mix=0.8478900852424016 ; last_regret=1.095589408195488
l_star=9 ; regret_mix=0.9181593238749907 ; last_regret=1.1636343292548086
l_star=13 ; regret_mix=0.8380031942976194 ; last_regret=1.034043883490907
l_star=10 ; regret_mix=0.897781341407361 ; last_regret=1.1043797315624582
l_star=12 ; regret_mix=0.8810174553020609 ; last_regret=1.1161179876473724
l_star=9 ; regret_mix=0.8752917916249358 ; last_regret=1.1121915722651068
l_star=10 ; regret_mix=0.8975885500521259 ; last_regret=1.1390563861500298
l_star=10 ; regret_mix=0.8738986507497619 ; last_regret=1.1996965458545508
 ***** Sigma=1.0 *****
l_star=7 ; regret_mix=1.154187242385968 ; last_regret=1.4423352502090732
l_star=9 ; regret_mix=1.0039086858318915 ; last_regret=1.1893068268157545
l_star=6 ; regret_mix=0.9763710859185195 ; last_regret=1.2385712

Exception ignored in: <_io.FileIO name='acc-ratio_vs_sigma0_0.5-10.0-0.5_opt-r=3_opt-sigma=5.p' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='sigma0-acc_0.5-10.0-0.5_opt-r=3_opt-sigma=5.p' mode='wb' closefd=True>
No handles with labels found to put in legend.


last_regret=3.146614700503987e-07


<Figure size 432x288 with 0 Axes>

In [12]:
# r
r_max = 10
r_min = 1
spaces = 1
load = False

suffix = f'{r_min}-{r_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    r_list = range(r_min, r_max, spaces)
    results = np.zeros([len(r_list), times])
    for idx, r in enumerate(r_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_r_.p', 'wb'))
    pickle.dump(r_list, open(f'r-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_r_{suffix}.p', 'rb'))
    r_list = pickle.load(open(f'r-acc_{suffix}.p', 'rb'))

plot_res(x_axis=r_list, results=results,
         x_label='r', y_label='Accuracy Ratio', label=f"d={d}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs r',
         path=f"results_accuracy_ratio_r_{suffix}")


l_star=2 ; regret_mix=0.9070362437226382 ; last_regret=0.9400492446699711
l_star=2 ; regret_mix=2.7696551210299725 ; last_regret=2.8209148396769614
l_star=2 ; regret_mix=3.2955605046257817 ; last_regret=3.3487057891737533
l_star=2 ; regret_mix=4.023268112306739 ; last_regret=4.086966903379754
l_star=2 ; regret_mix=0.050736776024568164 ; last_regret=0.05055405320384303
l_star=2 ; regret_mix=0.6670139763774006 ; last_regret=0.6802631296936584
l_star=2 ; regret_mix=0.10691647575756723 ; last_regret=0.1074039457909633
l_star=2 ; regret_mix=2.7868519198413546 ; last_regret=2.94194504660102
l_star=2 ; regret_mix=4.896108111195703 ; last_regret=4.998226623210869
l_star=2 ; regret_mix=4.0365017848736295 ; last_regret=4.073216146184629
l_star=3 ; regret_mix=0.26923749527947866 ; last_regret=0.28481492553252935
l_star=3 ; regret_mix=0.31659829004530216 ; last_regret=0.3437736228500176
l_star=3 ; regret_mix=1.1546644765938663 ; last_regret=1.3156574174568247
l_star=3 ; regret_mix=0.02426091576685

Exception ignored in: <_io.FileIO name='acc-ratio_vs_r_.p' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='r-acc_1-10-1_opt-r=3_opt-sigma=5.p' mode='wb' closefd=True>
No handles with labels found to put in legend.


last_regret=0.0005948031618093453


<Figure size 432x288 with 0 Axes>

In [13]:
# d
load = False
d_max = 20
d_min = opt_r+2
spaces = 1
suffix = f'{d_min}-{d_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    d_list = range(d_min, d_max, spaces)
    results = np.zeros([len(d_list), times])

    for idx, d in enumerate(d_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_d_{suffix}.p', 'wb'))
    pickle.dump(r_list, open(f'd-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_d_{suffix}.p', 'rb'))
    d_list = pickle.load(open(f'd-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_list, results=results,
         x_label='d', y_label='Accuracy Ratio', label=f"r={opt_r}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs d',
         path=f"results_accuracy_ratio_d_{suffix}")

l_star=4 ; regret_mix=0.001890503804680943 ; last_regret=0.00048034398736791886
l_star=4 ; regret_mix=0.0017754551064142357 ; last_regret=0.0002509965491396248
l_star=4 ; regret_mix=5.175878652225635e-05 ; last_regret=5.063802878797786e-05
l_star=5 ; regret_mix=0.0015047945314607073 ; last_regret=0.000540034481428997
l_star=4 ; regret_mix=0.001627331623804724 ; last_regret=0.00031840569213583954
l_star=4 ; regret_mix=0.001289175885486824 ; last_regret=0.00017440830300622065
l_star=4 ; regret_mix=0.004585666315027765 ; last_regret=0.0036694833833129867
l_star=4 ; regret_mix=0.03004047474169538 ; last_regret=0.03313206748128475
l_star=4 ; regret_mix=0.000142458042692065 ; last_regret=8.465054595741668e-05
l_star=4 ; regret_mix=1.0596719868036992e-06 ; last_regret=7.004422051703466e-06
l_star=4 ; regret_mix=1.88335812187022e-05 ; last_regret=4.7497600806307675e-06
l_star=4 ; regret_mix=0.023520099637852045 ; last_regret=0.02762039013356695
l_star=4 ; regret_mix=0.009787495799557674 ; last

KeyboardInterrupt: 

In [ ]:
# Regrets
load = False

if not load:
    d=20
    regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
    pickle.dump(regret_list, open(f'regrets.p', 'wb'))

else:
    regret_list = pickle.load(open(f'regrets.p', 'rb'))


plot_res(x_axis=range(m+2), results=regret_list.T,
         x_label='#Iteration', y_label='Regret', label=f'd={d} r={opt_r} '+r'$\sigma_0$='+f'{opt_sigma}',
         title='Regret vs #Iteration',
         path=f"Regret_vs_Iteration_r={opt_r}_var={opt_sigma}")
